In [5]:
import numpy as np
import scipy.special
import matplotlib.pyplot as plt

In [24]:
class neuralNetwork:
    
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrade):
        # задать количество узлов во входном, скрытом и выходном слое
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # коэффициент обучения
        self.lr = learningrade
        
        # Матрицы весовых коэффициентов связей wih (между входным и скрытым
        # слоями) и who (между скрытым и выходным слоями).
        # Весовые коэффициенты связей между узлом i и узлом j следующего слоя
        # обозначены как w_i_j:
        # wll w21
        # wl2 w22 и т.д.
        
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5),
                    (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5),
                    (self.onodes, self.hnodes))
    
    
        # использование сигмоиды в качестве функции активации
        self.activation_function = lambda x: scipy.special.expit(x)
        
    
    # тренировка нейронной сети
    def train(self, inputs_list, targets_list):
        # преобразовать список входных значений в двухмерный массив
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = np.dot(self.wih, inputs)
        # рассчитать иходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # рассчитать входящие сигналы для выходного слоя
        final_inputs = np.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        # ошибки выходного слоя = 
        # (целевое значение - фактическое значение)
        output_errors = targets - final_outputs
        # ошибки скрытого слоя - это ошибки output_errors,
        # распределенные пропорционально весовым коэффициентам связей
        # и рекомбинированные на скрытых узлах
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # обновить весовые коэффициенты для связей между
        # скрытым и выходным слоями
        self.who += self.lr * np.dot((output_errors * final_outputs *
                                     (1.0 - final_outputs)),
                                     np.transpose(hidden_outputs))
        
        # обновить весовые коэффициенты для связей между
        # входным и скрытым слоями
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs
                                     * (1.0 - hidden_outputs)), 
                                    np.transpose(inputs))
        
    
    # опрос нейронной сети
    def query(self, inputs_list):
        # преобразовать список входных значений
        # в двухмерный массив
        inputs = np.array(inputs_list, ndmin=2).T
        
        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = np.dot(self.wih, inputs)
        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # рассчитать входящие сигналы для выходного слоя
        final_inputs = np.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [31]:
# количество входных, скрытых и выходных узлов
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# коэффициент обучения 
learning_rate = 0.1

# создать экземпляр нейронной сети
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)

# загрузить в список тестовый набор данных CSV-файла набора MNIST
training_data_file = open("mnist_dataset/mnist_train_100.csv",)
training_data_list = training_data_file.readlines ()
training_data_file.close()
  
# тренировка нейронной сети
# переменная epochs указывает, сколько раз тренировочный
# набор данных используется для тренировки сети
epochs = 5
        
for e in range(epochs):
    # перебрать все записи в тренировачном наборе данных
    for record in training_data_list:
        # получить список значений из записи, используя символы
        # запятой (',f) в качестве разделителей
        all_values = record.split(',')
        # масштабировать и сместить входные значения
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # создать целевые выходные значения (все равны 0,01, за исключением
        # желаемого маркерного значения, равного 0,99)
        targets = np.zeros(output_nodes) + 0.01
        # all_values[0] - целевое маркерное значение для данной записи
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
    

In [32]:
# загрузить в список тестовый набор данных
# CSV-файла набора MNIST

test_data_file = open("mnist_dataset/mnist_test.csv")
test_data_list = test_data_file.readlines()
test_data_file.close()
                      
# тестирование нейронной сети
# журнал оценок работы сети, первоначально пустой
scorecard = []

# перебрать все записи в тестовом наборе
for record in test_data_list:
    # получить список значений из записи, используя символы
    # запятой (',*) в качестве разделителей
    all_values = record.split (',')
    # правильный ответ - первое значение
    correct_label = int(all_values[0])
    # # масштабировать и сместить входные значения
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # опрос сети
    outputs = n.query(inputs)
    # индекс наибольшего значения является маркерным значением
    label = np.argmax(outputs)
    # присоединить оценку ответа сети к концу списка
    if (label == correct_label):
        # в случае правильного ответа сети присоединить
        # к списку значение 1
        scorecard.append(1)
    else:
        # в случае неправильного ответа сети присоединить
        # к списку значение 0
        scorecard.append(0)
        
# # рассчитать показатель эффективности в виде
# доли правильных ответов

scorecard_array = np.asarray(scorecard)
print("Эффективность = ", scorecard_array.sum() / scorecard_array.size)

Эффективность =  0.6718
